### Install dependencies

In [ ]:
!pip install opencv-python==3.4.2.16
!pip install imutils
!pip install skimage
!pip install matplotlib
!pip install numpy

### Import packages

In [44]:
import cv2
from skimage.measure import compare_ssim
import imutils
import matplotlib.pyplot as plt
import numpy as np
import os
%matplotlib inline

In [51]:
#image taken as background

#read image and video
#img_dir = './datasets/CDnet2014/dataset/baseline/PETS2006/input/'
img_dir = './datasets/CDnet2014/dataset/cameraJitter/boulevard/input/'
video_data = []
n_frames=0
for f1 in os.listdir(img_dir):
    img = cv2.imread(os.path.join(img_dir,f1))
    video_data.append(img)
    n_frames+=1

In [52]:
#img_original = cv2.cvtColor(cv2.imread(os.path.join(img_dir,'in000001.jpg')), cv2.COLOR_BGR2GRAY)
img_original = cv2.imread(os.path.join(img_dir,'in000001.jpg'))
img_original_shape = img_original.shape
h=img_original.shape[0]
w=img_original.shape[1]
fgbg = cv2.createBackgroundSubtractorMOG2()

#outputing video
#fourcc = cv2.VideoWriter_fourcc(*'MJPG')
#out = cv2.VideoWriter('video_out.avi', fourcc, 30,(w,h))

In [55]:
#find difference in frames
for frame in video_data:
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame = cv2.GaussianBlur(frame,(5,5),0)
    (score, diff) = compare_ssim(frame, cv2.GaussianBlur(cv2.cvtColor(img_original, cv2.COLOR_BGR2GRAY),(5,5),0), full=True)
    diff = (diff * 255).astype("uint8")
    #print(score, end=" ")
    
    mask = np.zeros(img_original.shape[:2], dtype = "uint8")
    fgmask = fgbg.apply(frame)
    #fgmask_masked = cv2.bitwise_and(fgmask,fgmask,mask=mask)
    
    thresh = cv2.threshold(diff, 0, 255,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
    #thresh = cv2.adaptiveThreshold(diff,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,11,2)[1]
    
    #img_diff=abs(img_original-frame)
    cv2.imshow("Diff", diff)
    cv2.imshow("Thresh", thresh)
    cv2.imshow("fgmask", fgmask)
    #cv2.imshow("fgmasked", fgmask_masked)
    keyboard = cv2.waitKey(30) & 0xFF
    if keyboard == 'q' or keyboard == 27:
        break
cv2.destroyAllWindows()

In [54]:
prev_gray=cv2.cvtColor(img_original, cv2.COLOR_BGR2GRAY)
# Pre-define transformation-store array
transforms = np.zeros((n_frames-1, 3), np.float32) 
 
for i in range(n_frames-2):
  # Detect feature points in previous frame
  prev_pts = cv2.goodFeaturesToTrack(prev_gray,
                                     maxCorners=200,
                                     qualityLevel=0.01,
                                     minDistance=30,
                                     blockSize=3)
    
  # Read next frame
  curr = video_data[i+1]
 
  # Convert to grayscale
  curr_gray = cv2.cvtColor(curr, cv2.COLOR_BGR2GRAY) 
 
  # Calculate optical flow (i.e. track feature points)
  curr_pts, status, err = cv2.calcOpticalFlowPyrLK(prev_gray, curr_gray, prev_pts, None) 
 
  # Sanity check
  assert prev_pts.shape == curr_pts.shape 
 
  # Filter only valid points
  idx = np.where(status==1)[0]
  prev_pts = prev_pts[idx]
  curr_pts = curr_pts[idx]
 
  #Find transformation matrix
  m = cv2.estimateRigidTransform(prev_pts, curr_pts, fullAffine=False) #will only work with OpenCV-3 or less
    
  # Extract traslation
  dx = m[0,2]
  dy = m[1,2]
 
  # Extract rotation angle
  da = np.arctan2(m[1,0], m[0,0])
    
  # Store transformation
  transforms[i] = [dx,dy,da]
    
  # Move to next frame
  prev_gray = curr_gray
 
  print("Frame: " + str(i) +  "/" + str(n_frames) + " -  Tracked points : " + str(len(prev_pts)))

Frame: 0/2500 -  Tracked points : 51
Frame: 1/2500 -  Tracked points : 43
Frame: 2/2500 -  Tracked points : 41
Frame: 3/2500 -  Tracked points : 53
Frame: 4/2500 -  Tracked points : 49
Frame: 5/2500 -  Tracked points : 48
Frame: 6/2500 -  Tracked points : 49
Frame: 7/2500 -  Tracked points : 52
Frame: 8/2500 -  Tracked points : 51
Frame: 9/2500 -  Tracked points : 49
Frame: 10/2500 -  Tracked points : 42
Frame: 11/2500 -  Tracked points : 49
Frame: 12/2500 -  Tracked points : 48
Frame: 13/2500 -  Tracked points : 44
Frame: 14/2500 -  Tracked points : 41
Frame: 15/2500 -  Tracked points : 44
Frame: 16/2500 -  Tracked points : 45
Frame: 17/2500 -  Tracked points : 45
Frame: 18/2500 -  Tracked points : 42
Frame: 19/2500 -  Tracked points : 41
Frame: 20/2500 -  Tracked points : 49
Frame: 21/2500 -  Tracked points : 44
Frame: 22/2500 -  Tracked points : 47
Frame: 23/2500 -  Tracked points : 44
Frame: 24/2500 -  Tracked points : 39
Frame: 25/2500 -  Tracked points : 40
Frame: 26/2500 -  Trac

Frame: 216/2500 -  Tracked points : 50
Frame: 217/2500 -  Tracked points : 51
Frame: 218/2500 -  Tracked points : 50
Frame: 219/2500 -  Tracked points : 49
Frame: 220/2500 -  Tracked points : 49
Frame: 221/2500 -  Tracked points : 45
Frame: 222/2500 -  Tracked points : 50
Frame: 223/2500 -  Tracked points : 43
Frame: 224/2500 -  Tracked points : 47
Frame: 225/2500 -  Tracked points : 46
Frame: 226/2500 -  Tracked points : 45
Frame: 227/2500 -  Tracked points : 43
Frame: 228/2500 -  Tracked points : 47
Frame: 229/2500 -  Tracked points : 43
Frame: 230/2500 -  Tracked points : 39
Frame: 231/2500 -  Tracked points : 38
Frame: 232/2500 -  Tracked points : 45
Frame: 233/2500 -  Tracked points : 49
Frame: 234/2500 -  Tracked points : 45
Frame: 235/2500 -  Tracked points : 48
Frame: 236/2500 -  Tracked points : 45
Frame: 237/2500 -  Tracked points : 38
Frame: 238/2500 -  Tracked points : 38
Frame: 239/2500 -  Tracked points : 42
Frame: 240/2500 -  Tracked points : 44
Frame: 241/2500 -  Tracke

Frame: 442/2500 -  Tracked points : 46
Frame: 443/2500 -  Tracked points : 40
Frame: 444/2500 -  Tracked points : 40
Frame: 445/2500 -  Tracked points : 44
Frame: 446/2500 -  Tracked points : 42
Frame: 447/2500 -  Tracked points : 37
Frame: 448/2500 -  Tracked points : 43
Frame: 449/2500 -  Tracked points : 47
Frame: 450/2500 -  Tracked points : 38
Frame: 451/2500 -  Tracked points : 43
Frame: 452/2500 -  Tracked points : 51
Frame: 453/2500 -  Tracked points : 40
Frame: 454/2500 -  Tracked points : 42
Frame: 455/2500 -  Tracked points : 43
Frame: 456/2500 -  Tracked points : 49
Frame: 457/2500 -  Tracked points : 48
Frame: 458/2500 -  Tracked points : 49
Frame: 459/2500 -  Tracked points : 52
Frame: 460/2500 -  Tracked points : 51
Frame: 461/2500 -  Tracked points : 50
Frame: 462/2500 -  Tracked points : 50
Frame: 463/2500 -  Tracked points : 52
Frame: 464/2500 -  Tracked points : 46
Frame: 465/2500 -  Tracked points : 48
Frame: 466/2500 -  Tracked points : 48
Frame: 467/2500 -  Tracke

Frame: 675/2500 -  Tracked points : 52
Frame: 676/2500 -  Tracked points : 49
Frame: 677/2500 -  Tracked points : 43
Frame: 678/2500 -  Tracked points : 43
Frame: 679/2500 -  Tracked points : 48
Frame: 680/2500 -  Tracked points : 43
Frame: 681/2500 -  Tracked points : 37
Frame: 682/2500 -  Tracked points : 41
Frame: 683/2500 -  Tracked points : 42
Frame: 684/2500 -  Tracked points : 35
Frame: 685/2500 -  Tracked points : 35
Frame: 686/2500 -  Tracked points : 42
Frame: 687/2500 -  Tracked points : 38
Frame: 688/2500 -  Tracked points : 34
Frame: 689/2500 -  Tracked points : 40
Frame: 690/2500 -  Tracked points : 44
Frame: 691/2500 -  Tracked points : 36
Frame: 692/2500 -  Tracked points : 38
Frame: 693/2500 -  Tracked points : 40
Frame: 694/2500 -  Tracked points : 41
Frame: 695/2500 -  Tracked points : 35
Frame: 696/2500 -  Tracked points : 38
Frame: 697/2500 -  Tracked points : 43
Frame: 698/2500 -  Tracked points : 36
Frame: 699/2500 -  Tracked points : 38
Frame: 700/2500 -  Tracke

Frame: 909/2500 -  Tracked points : 41
Frame: 910/2500 -  Tracked points : 39
Frame: 911/2500 -  Tracked points : 47
Frame: 912/2500 -  Tracked points : 46
Frame: 913/2500 -  Tracked points : 41
Frame: 914/2500 -  Tracked points : 48
Frame: 915/2500 -  Tracked points : 50
Frame: 916/2500 -  Tracked points : 47
Frame: 917/2500 -  Tracked points : 47
Frame: 918/2500 -  Tracked points : 47
Frame: 919/2500 -  Tracked points : 51
Frame: 920/2500 -  Tracked points : 51
Frame: 921/2500 -  Tracked points : 53
Frame: 922/2500 -  Tracked points : 50
Frame: 923/2500 -  Tracked points : 43
Frame: 924/2500 -  Tracked points : 50
Frame: 925/2500 -  Tracked points : 51
Frame: 926/2500 -  Tracked points : 44
Frame: 927/2500 -  Tracked points : 41
Frame: 928/2500 -  Tracked points : 45
Frame: 929/2500 -  Tracked points : 49
Frame: 930/2500 -  Tracked points : 41
Frame: 931/2500 -  Tracked points : 44
Frame: 932/2500 -  Tracked points : 51
Frame: 933/2500 -  Tracked points : 44
Frame: 934/2500 -  Tracke

...

Frame: 1578/2500 -  Tracked points : 46
Frame: 1579/2500 -  Tracked points : 41
Frame: 1580/2500 -  Tracked points : 38
Frame: 1581/2500 -  Tracked points : 44
Frame: 1582/2500 -  Tracked points : 48
Frame: 1583/2500 -  Tracked points : 41
Frame: 1584/2500 -  Tracked points : 40
Frame: 1585/2500 -  Tracked points : 48
Frame: 1586/2500 -  Tracked points : 42
Frame: 1587/2500 -  Tracked points : 39
Frame: 1588/2500 -  Tracked points : 43
Frame: 1589/2500 -  Tracked points : 45
Frame: 1590/2500 -  Tracked points : 35
Frame: 1591/2500 -  Tracked points : 38
Frame: 1592/2500 -  Tracked points : 46
Frame: 1593/2500 -  Tracked points : 40
Frame: 1594/2500 -  Tracked points : 40
Frame: 1595/2500 -  Tracked points : 43
Frame: 1596/2500 -  Tracked points : 45
Frame: 1597/2500 -  Tracked points : 37
Frame: 1598/2500 -  Tracked points : 38
Frame: 1599/2500 -  Tracked points : 47
Frame: 1600/2500 -  Tracked points : 38
Frame: 1601/2500 -  Tracked points : 39
Frame: 1602/2500 -  Tracked points : 43


Frame: 1801/2500 -  Tracked points : 37
Frame: 1802/2500 -  Tracked points : 40
Frame: 1803/2500 -  Tracked points : 41
Frame: 1804/2500 -  Tracked points : 36
Frame: 1805/2500 -  Tracked points : 39
Frame: 1806/2500 -  Tracked points : 44
Frame: 1807/2500 -  Tracked points : 37
Frame: 1808/2500 -  Tracked points : 38
Frame: 1809/2500 -  Tracked points : 43
Frame: 1810/2500 -  Tracked points : 43
Frame: 1811/2500 -  Tracked points : 41
Frame: 1812/2500 -  Tracked points : 43
Frame: 1813/2500 -  Tracked points : 43
Frame: 1814/2500 -  Tracked points : 43
Frame: 1815/2500 -  Tracked points : 41
Frame: 1816/2500 -  Tracked points : 45
Frame: 1817/2500 -  Tracked points : 42
Frame: 1818/2500 -  Tracked points : 45
Frame: 1819/2500 -  Tracked points : 46
Frame: 1820/2500 -  Tracked points : 43
Frame: 1821/2500 -  Tracked points : 39
Frame: 1822/2500 -  Tracked points : 42
Frame: 1823/2500 -  Tracked points : 47
Frame: 1824/2500 -  Tracked points : 40
Frame: 1825/2500 -  Tracked points : 45


Frame: 2033/2500 -  Tracked points : 44
Frame: 2034/2500 -  Tracked points : 48
Frame: 2035/2500 -  Tracked points : 47
Frame: 2036/2500 -  Tracked points : 42
Frame: 2037/2500 -  Tracked points : 41
Frame: 2038/2500 -  Tracked points : 45
Frame: 2039/2500 -  Tracked points : 44
Frame: 2040/2500 -  Tracked points : 39
Frame: 2041/2500 -  Tracked points : 38
Frame: 2042/2500 -  Tracked points : 43
Frame: 2043/2500 -  Tracked points : 43
Frame: 2044/2500 -  Tracked points : 35
Frame: 2045/2500 -  Tracked points : 42
Frame: 2046/2500 -  Tracked points : 43
Frame: 2047/2500 -  Tracked points : 37
Frame: 2048/2500 -  Tracked points : 37
Frame: 2049/2500 -  Tracked points : 45
Frame: 2050/2500 -  Tracked points : 44
Frame: 2051/2500 -  Tracked points : 39
Frame: 2052/2500 -  Tracked points : 38
Frame: 2053/2500 -  Tracked points : 48
Frame: 2054/2500 -  Tracked points : 37
Frame: 2055/2500 -  Tracked points : 40
Frame: 2056/2500 -  Tracked points : 43
Frame: 2057/2500 -  Tracked points : 43


Frame: 2266/2500 -  Tracked points : 42
Frame: 2267/2500 -  Tracked points : 43
Frame: 2268/2500 -  Tracked points : 48
Frame: 2269/2500 -  Tracked points : 43
Frame: 2270/2500 -  Tracked points : 43
Frame: 2271/2500 -  Tracked points : 45
Frame: 2272/2500 -  Tracked points : 43
Frame: 2273/2500 -  Tracked points : 44
Frame: 2274/2500 -  Tracked points : 47
Frame: 2275/2500 -  Tracked points : 46
Frame: 2276/2500 -  Tracked points : 46
Frame: 2277/2500 -  Tracked points : 46
Frame: 2278/2500 -  Tracked points : 45
Frame: 2279/2500 -  Tracked points : 47
Frame: 2280/2500 -  Tracked points : 49
Frame: 2281/2500 -  Tracked points : 45
Frame: 2282/2500 -  Tracked points : 42
Frame: 2283/2500 -  Tracked points : 48
Frame: 2284/2500 -  Tracked points : 50
Frame: 2285/2500 -  Tracked points : 47
Frame: 2286/2500 -  Tracked points : 45
Frame: 2287/2500 -  Tracked points : 52
Frame: 2288/2500 -  Tracked points : 47
Frame: 2289/2500 -  Tracked points : 46
Frame: 2290/2500 -  Tracked points : 53


Frame: 2484/2500 -  Tracked points : 44
Frame: 2485/2500 -  Tracked points : 42
Frame: 2486/2500 -  Tracked points : 41
Frame: 2487/2500 -  Tracked points : 44
Frame: 2488/2500 -  Tracked points : 48
Frame: 2489/2500 -  Tracked points : 43
Frame: 2490/2500 -  Tracked points : 44
Frame: 2491/2500 -  Tracked points : 42
Frame: 2492/2500 -  Tracked points : 42
Frame: 2493/2500 -  Tracked points : 44
Frame: 2494/2500 -  Tracked points : 42
Frame: 2495/2500 -  Tracked points : 41
Frame: 2496/2500 -  Tracked points : 47
Frame: 2497/2500 -  Tracked points : 43


In [27]:
def movingAverage(curve, radius): 
  window_size = 2 * radius + 1
  # Define the filter 
  f = np.ones(window_size)/window_size 
  # Add padding to the boundaries 
  curve_pad = np.lib.pad(curve, (radius, radius), 'edge') 
  # Apply convolution 
  curve_smoothed = np.convolve(curve_pad, f, mode='same') 
  # Remove padding 
  curve_smoothed = curve_smoothed[radius:-radius]
  # return smoothed curve
  return curve_smoothed 

def smooth(trajectory): 
  smoothed_trajectory = np.copy(trajectory) 
  SMOOTHING_RADIUS=5
  # Filter the x, y and angle curves
  for i in range(3):
    smoothed_trajectory[:,i] = movingAverage(trajectory[:,i], radius=SMOOTHING_RADIUS)
  return smoothed_trajectory

def fixBorder(frame):
  s = frame.shape
  # Scale the image 4% without moving the center
  T = cv2.getRotationMatrix2D((s[1]/2, s[0]/2), 0, 1.04)
  frame = cv2.warpAffine(frame, T, (s[1], s[0]))
  return frame

In [28]:
# Compute trajectory using cumulative sum of transformations
trajectory = np.cumsum(transforms, axis=0)

# Calculate difference in smoothed_trajectory and trajectory
difference = smooth(trajectory) - trajectory
  
# Calculate newer transformation array
transforms_smooth = transforms + difference

In [30]:
# Write n_frames-1 transformed frames
for i in range(n_frames-2):
  # Read next frame
  frame = video_data[i+1]
 
  # Extract transformations from the new transformation array
  dx = transforms_smooth[i,0]
  dy = transforms_smooth[i,1]
  da = transforms_smooth[i,2]
 
  # Reconstruct transformation matrix accordingly to new values
  m = np.zeros((2,3), np.float32)
  m[0,0] = np.cos(da)
  m[0,1] = -np.sin(da)
  m[1,0] = np.sin(da)
  m[1,1] = np.cos(da)
  m[0,2] = dx
  m[1,2] = dy
 
  # Apply affine wrapping to the given frame
  frame_stabilized = cv2.warpAffine(frame, m, (h,w))
  # Fix border artifacts
  frame_stabilized = fixBorder(frame_stabilized) 
 
  # Write the frame to the file
  frame_out = cv2.hconcat([frame, frame_stabilized])
 
  # If the image is too big, resize it.
  if(frame_out.shape[1] > 1920): 
    frame_out = cv2.resize(frame_out, (frame_out.shape[1]/2, frame_out.shape[0]/2));
   
  cv2.imshow("Before and After", frame_out)
  cv2.waitKey(10)
  out.write(frame_out)
cv2.destroyAllWindows()

In [56]:
#out.release()